In [ ]:
import pvdeg
from pvdeg.geospatial import output_template
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import pandas as pd
import xarray as xr
import dask.array as da
import pickle

from dask.distributed import LocalCluster, Client

In [ ]:
workers = 8

cluster = LocalCluster(
    n_workers=workers,
    processes=True, 
    )

client = Client(cluster)

print(client.dashboard_link)

In [ ]:
locationGetter = pvdeg.scenario.GeospatialScenario()

locationGetter.addLocation(country="United States", downsample_factor=40)

In [ ]:
locationGetter.plot_coords()

In [ ]:
geo_weather, geo_meta = locationGetter.geospatial_data()

geo_meta = geo_meta[geo_meta['state'] != "Alaska"]
geo_weather = geo_weather.sel(gid=geo_meta.index)

In [ ]:
shapes = {
    "annual_poa" : ("gid",),
    "ground_irradiance" : ("gid", "time", "distance")
}

template = pvdeg.geospatial.output_template(
    ds_gids = geo_weather,
    shapes = shapes,
    add_dims={"distance" : 10}, # this will autogenerate a range of length 10 for the coordinate axis
)

In [ ]:
pysam_tiny_res = pvdeg.geospatial.analysis(
    weather_ds = geo_weather,
    meta_df = geo_meta,
    func = pvdeg.pysam.inspire_ground_irradiance, 
    template = template,
    config_files = {"pv" : '/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json'}
)

In [ ]:
pysam_tiny_res.to_netcdf('downsample40-conf01-pysam-ground-irradiance.nc', engine='h5netcdf')

In [ ]:
pysam_tiny_res

In [ ]:
res = pvdeg.pysam.inspire_ground_irradiance(
    geo_weather.isel(gid=0).to_dataframe(),
    geo_meta.iloc[0].to_dict(),
    config_files = {"pv" : '/home/tford/dev/InSPIRE/Studies/USMap_Doubleday_2024/SAM/01/01_pvsamv1.json'}
)

In [ ]:
res.ground_irradiance[0][1:]